# Simple Spark Parallelisation Test

This notebook connects to a Spark cluster, configures the NVIDIA Accelerator for Apache Spark, and runs a simple GPU-accelerated workload by joining two dataframes.

Make sure you set the correct Spark configuration to reflect the capacity of the on-demand Spark cluster that you've attached to this Workspace. Important parameters to configure below are:

* spark.executor.resource.gpu.amount
* spark.executor.cores
* spark.task.resource.gpu.amount
* spark.rapids.sql.concurrentGpuTasks

Note that spark.task.resource.gpu.amount can be a decimal amount, so if you want multiple tasks to be run on an executor at the same time and assigned to the same GPU you can set this to a decimal value less than 1. You would want this setting to correspond to the spark.executor.cores setting. For instance, if you have spark.executor.cores=2 which would allow 2 tasks to run on each executor and you want those 2 tasks to run on the same GPU then you would set spark.task.resource.gpu.amount=0.5. See the Tuning Guide for more details on controlling the task concurrency for each executor.

See the [RAPIDS Accelerator for Apache Spark Tuning Guide](https://nvidia.github.io/spark-rapids/docs/tuning-guide.html) for additional details.


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
.config("spark.task.cpus", 1) \
.config("spark.driver.extraClassPath", "/opt/sparkRapidsPlugin/rapids-4-spark_2.12-0.1.0.jar:/opt/sparkRapidsPlugin/cudf-0.14-cuda10-1.jar") \
.config("spark.executor.extraClassPath", "/opt/sparkRapidsPlugin/rapids-4-spark_2.12-0.1.0.jar:/opt/sparkRapidsPlugin/cudf-0.14-cuda10-1.jar") \
.config("spark.executor.resource.gpu.amount", 1) \
.config("spark.executor.cores", 6) \
.config("spark.task.resource.gpu.amount", 0.15) \
.config("spark.rapids.sql.concurrentGpuTasks", 1) \
.config("spark.rapids.memory.pinnedPool.size", "2G") \
.config("spark.locality.wait", "0s") \
.config("spark.sql.files.maxPartitionBytes", "512m") \
.config("spark.sql.shuffle.partitions", 10) \
.config("spark.plugins", "com.nvidia.spark.SQLPlugin") \
.appName("MyGPUAppName") \
.getOrCreate()

Create and run the workload.

In [ ]:
df1 = spark.sparkContext.parallelize(range(1, 100)).map(lambda x: (x, "a" * x)).toDF()
df2 = spark.sparkContext.parallelize(range(1, 100)).map(lambda x: (x, "b" * x)).toDF()
df = df1.join(df2, how="outer")
df.count()

Check the Spark WebUI console and confirm that the default Spark operations in the DAG have been replaced with GPU-accelerated versions. When done run the next cell to stop the test application and release its resources back to the cluster.

In [ ]:
spark.stop()